In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time


In [2]:
#API KEYS
bls_api = '476d935074304bb5921d6c11f03abcc3'
bea_api = '1877DB8B-6730-4F80-B187-242ACB99D57C'
bls_api_2 = 'f2476fb2c0584b3c910934b13ca2aa1a'

In [3]:
#URLS

url_bea = "https://apps.bea.gov/api/data"
url_bls = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [4]:
#Datos para demsempleo por Estado
estados = {
    "Alabama": "LAUST010000000000003",
    "Alaska": "LAUST020000000000003",
    "Arizona": "LAUST040000000000003",
    "Arkansas": "LAUST050000000000003",
    "California": "LAUST060000000000003",
    "Colorado": "LAUST080000000000003",
    "Connecticut": "LAUST090000000000003",
    "Delaware": "LAUST100000000000003",
    "Florida": "LAUST120000000000003",
    "Georgia": "LAUST130000000000003",
    "Hawaii": "LAUST150000000000003",
    "Idaho": "LAUST160000000000003",
    "Illinois": "LAUST170000000000003",
    "Indiana": "LAUST180000000000003",
    "Iowa": "LAUST190000000000003",
    "Kansas": "LAUST200000000000003",
    "Kentucky": "LAUST210000000000003",
    "Louisiana": "LAUST220000000000003",
    "Maine": "LAUST230000000000003",
    "Maryland": "LAUST240000000000003",
    "Massachusetts": "LAUST250000000000003",
    "Michigan": "LAUST260000000000003",
    "Minnesota": "LAUST270000000000003",
    "Mississippi": "LAUST280000000000003",
    "Missouri": "LAUST290000000000003",
    "Montana": "LAUST300000000000003",
    "Nebraska": "LAUST310000000000003",
    "Nevada": "LAUST320000000000003",
    "New Hampshire": "LAUST330000000000003",
    "New Jersey": "LAUST340000000000003",
    "New Mexico": "LAUST350000000000003",
    "New York": "LAUST360000000000003",
    "North Carolina": "LAUST370000000000003",
    "North Dakota": "LAUST380000000000003",
    "Ohio": "LAUST390000000000003",
    "Oklahoma": "LAUST400000000000003",
    "Oregon": "LAUST410000000000003",
    "Pennsylvania": "LAUST420000000000003",
    "Rhode Island": "LAUST440000000000003",
    "South Carolina": "LAUST450000000000003",
    "South Dakota": "LAUST460000000000003",
    "Tennessee": "LAUST470000000000003",
    "Texas": "LAUST480000000000003",
    "Utah": "LAUST490000000000003",
    "Vermont": "LAUST500000000000003",
    "Virginia": "LAUST510000000000003",
    "Washington": "LAUST530000000000003",
    "West Virginia": "LAUST540000000000003",
    "Wisconsin": "LAUST550000000000003",
    "Wyoming": "LAUST560000000000003"
}

In [5]:
meses = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

In [6]:
rangos = [(1990,1999),(2000, 2009), (2010, 2019), (2020, 2025)]

In [7]:
#Payload
registros = []

series = list(estados.values())

for start, end in rangos:
    for i in range(0, len(series), 25):  
        bloque = series[i:i+25]
        payload = {
            "registrationKey": bls_api,
            "seriesid": bloque,
            "startyear": str(start),
            "endyear": str(end)
        }
        url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
        response = requests.post(url, json=payload)
        data = response.json()

        # Procesar cada serie
        for serie in data["Results"]["series"]:
            estado = [k for k, v in estados.items() if v == serie["seriesID"]][0]
            for d in serie["data"]:
                if d["periodName"] in meses:
                    registros.append({
                        "Estado": estado,
                        "Año": int(d["year"]),
                        "Mes": meses[d["periodName"]],
                        "TasaDesempleo": float(d["value"])
                    })
        time.sleep(0.5)  # pequeño delay para no saturar la API


In [8]:
df = pd.DataFrame(registros)
df["Fecha"] = pd.to_datetime(df["Año"].astype(str) + "-" + df["Mes"].astype(str).str.zfill(2))
df = df.sort_values(["Estado", "Fecha"]).reset_index(drop=True)
df = df[df['Estado'] != 'New York city']
df = df[df['Estado'] != 'Los Angeles County']
df = df[['Estado','Fecha','TasaDesempleo']]
df['Trim'] = df['Fecha'].dt.to_period('Q').astype(str)
df

,Estado,Fecha,TasaDesempleo,Trim
0,Alabama,1990-01-01,7.1,1990Q1
1,Alabama,1990-02-01,7.3,1990Q1
2,Alabama,1990-03-01,6.6,1990Q1
3,Alabama,1990-04-01,6.4,1990Q2
4,Alabama,1990-05-01,6.3,1990Q2
...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2
21396,Wyoming,2025-05-01,3.2,2025Q2
21397,Wyoming,2025-06-01,3.7,2025Q2
21398,Wyoming,2025-07-01,2.8,2025Q3


In [9]:
#Parametros para GDP

params = {
    "UserID": bea_api,
    "method": "GetData",
    "datasetname": "Regional",
    "TableName": "SQGDP1",   
    "Frequency": "Q",
    "Year": "ALL",    
    'GeoFips':'STATE',     
    "ResultFormat": "JSON",
    'LineCode':1
}

In [10]:
#Request GDP
response = requests.get(url_bea, params=params)
response.status_code == 200

True

In [11]:
gdp = response.json()
gdp

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': '1877DB8B-6730-4F80-B187-242ACB99D57C'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQGDP1'},
    {'ParameterName': 'FREQUENCY', 'ParameterValue': 'Q'},
    {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'}]},
  'Results': {'Statistic': 'Real Gross Domestic Product (GDP)',
   'UnitOfMeasure': 'Millions of chained 2017 dollars',
   'PublicTable': 'SQGDP1 State quarterly gross domestic product (GDP) summary',
   'UTCProductionTime': '2025-10-30T15:01:43.100',
   'NoteRef': '1',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'Da

In [12]:
records = gdp['BEAAPI']['Results']['Data']
records

[{'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q1',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '15844727.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q2',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '15922782.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q3',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '16047587.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q4',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '16136734.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2006Q1',
  'CL_UNIT': 'Millions of chaine

In [13]:
gdp_data = pd.DataFrame(records)[['GeoName','TimePeriod','DataValue']]
gdp_data[gdp_data['GeoName']=='United States']
gdp_data

,GeoName,TimePeriod,DataValue
0,United States,2005Q1,15844727.0
1,United States,2005Q2,15922782.0
2,United States,2005Q3,16047587.0
3,United States,2005Q4,16136734.0
4,United States,2006Q1,16353835.0
...,...,...,...
4915,Far West,2024Q2,4606652.1
4916,Far West,2024Q3,4649588.3
4917,Far West,2024Q4,4679775.7
4918,Far West,2025Q1,4674104.5


In [14]:
gdp_data["DataValue"] = pd.to_numeric(gdp_data["DataValue"], errors="coerce")
gdp_data.rename(columns={'GeoName':'Estado','TimePeriod':'Trim','DataValue':'GDP'}, inplace=True)
gdp_data

,Estado,Trim,GDP
0,United States,2005Q1,15844727.0
1,United States,2005Q2,15922782.0
2,United States,2005Q3,16047587.0
3,United States,2005Q4,16136734.0
4,United States,2006Q1,16353835.0
...,...,...,...
4915,Far West,2024Q2,4606652.1
4916,Far West,2024Q3,4649588.3
4917,Far West,2024Q4,4679775.7
4918,Far West,2025Q1,4674104.5


In [15]:
base_cox = pd.merge(
    df,
    gdp_data[['Estado','Trim','GDP']],
    on=['Estado','Trim'],
    how='left'
)
base_cox

,Estado,Fecha,TasaDesempleo,Trim,GDP
0,Alabama,1990-01-01,7.1,1990Q1,NaN
1,Alabama,1990-02-01,7.3,1990Q1,NaN
2,Alabama,1990-03-01,6.6,1990Q1,NaN
3,Alabama,1990-04-01,6.4,1990Q2,NaN
4,Alabama,1990-05-01,6.3,1990Q2,NaN
...,...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2,39982.7
21396,Wyoming,2025-05-01,3.2,2025Q2,39982.7
21397,Wyoming,2025-06-01,3.7,2025Q2,39982.7
21398,Wyoming,2025-07-01,2.8,2025Q3,NaN


In [16]:
#Parametros Personla Income
params_pi = {
    "UserID": bea_api,
    "method": "GetData",
    "datasetname": "Regional",
    "TableName": "SQINC1",   
    "Frequency": "Q",
    "Year": "ALL",    
    'GeoFips':'STATE',     
    "ResultFormat": "JSON",
    'LineCode':1
}

In [17]:
#Request 
response_pi = requests.get(url_bea, params=params_pi)
response_pi.status_code == 200

True

In [18]:
pi = response_pi.json()
pi

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': '1877DB8B-6730-4F80-B187-242ACB99D57C'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQINC1'},
    {'ParameterName': 'FREQUENCY', 'ParameterValue': 'Q'},
    {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'}]},
  'Results': {'Statistic': 'Personal income',
   'UnitOfMeasure': 'Millions of dollars',
   'PublicTable': 'SQINC1 State quarterly personal income summary: personal income, population, per capita personal income',
   'UTCProductionTime': '2025-10-30T15:01:46.263',
   'NoteRef': ' ',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': '

In [19]:
records_pi = pi['BEAAPI']['Results']['Data']
records_pi

[{'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q1',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '204641.7'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q2',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '210069.4'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q3',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '215368.6'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q4',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '215900.3'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1949Q1',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '209582.6'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'Tim

In [20]:
data_pi = pd.DataFrame(records_pi)[['GeoName','TimePeriod','DataValue']]
data_pi

,GeoName,TimePeriod,DataValue
0,United States,1948Q1,204641.7
1,United States,1948Q2,210069.4
2,United States,1948Q3,215368.6
3,United States,1948Q4,215900.3
4,United States,1949Q1,209582.6
...,...,...,...
18595,Far West *,1991Q2,889201.1
18596,Far West *,1991Q3,898644.1
18597,Far West *,1991Q4,912345.6
18598,Far West *,1992Q1,928602.4


In [21]:
data_pi['DataValue'] = pd.to_numeric(data_pi['DataValue'],errors='coerce')
data_pi.rename(columns={'GeoName':'Estado','TimePeriod':'Trim','DataValue':'Personal_Income'}, inplace=True)
data_pi["Estado"] = data_pi["Estado"].str.replace("*", "", regex=False).str.strip()
data_pi


,Estado,Trim,Personal_Income
0,United States,1948Q1,204641.7
1,United States,1948Q2,210069.4
2,United States,1948Q3,215368.6
3,United States,1948Q4,215900.3
4,United States,1949Q1,209582.6
...,...,...,...
18595,Far West,1991Q2,889201.1
18596,Far West,1991Q3,898644.1
18597,Far West,1991Q4,912345.6
18598,Far West,1992Q1,928602.4


In [22]:
base_cox = pd.merge(
    base_cox,
    data_pi[['Estado','Trim','Personal_Income']],
    on=['Estado','Trim'],
    how='left'
)

In [23]:
#Separacion por zonas
state_abbr = {
    "Alabama":"AL", "Alaska":"AK", "Arizona":"AZ", "Arkansas":"AR", "California":"CA",
    "Colorado":"CO", "Connecticut":"CT", "Delaware":"DE", "DistrictOfColumbia":"DC",
    "Florida":"FL", "Georgia":"GA", "Hawaii":"HI", "Idaho":"ID", "Illinois":"IL",
    "Indiana":"IN", "Iowa":"IA", "Kansas":"KS", "Kentucky":"KY", "Louisiana":"LA",
    "Maine":"ME", "Maryland":"MD", "Massachusetts":"MA", "Michigan":"MI",
    "Minnesota":"MN", "Mississippi":"MS", "Missouri":"MO", "Montana":"MT",
    "Nebraska":"NE", "Nevada":"NV", "NewHampshire":"NH", "NewJersey":"NJ",
    "NewMexico":"NM", "NewYork":"NY", "NorthCarolina":"NC", "NorthDakota":"ND",
    "Ohio":"OH", "Oklahoma":"OK", "Oregon":"OR", "Pennsylvania":"PA",
    "RhodeIsland":"RI", "SouthCarolina":"SC", "SouthDakota":"SD", "Tennessee":"TN",
    "Texas":"TX", "Utah":"UT", "Vermont":"VT", "Virginia":"VA", "Washington":"WA",
    "WestVirginia":"WV", "Wisconsin":"WI", "Wyoming":"WY"
}

state_region_map = {
    # --- NORTE ---
    "ME": "Norte", "NH": "Norte", "VT": "Norte", "MA": "Norte", "RI": "Norte", "CT": "Norte",
    "NY": "Norte", "NJ": "Norte", "PA": "Norte", "OH": "Norte", "MI": "Norte", "WI": "Norte",
    "MN": "Norte", "ND": "Norte", "SD": "Norte",

    # --- ESTE ---
    "DE": "Este", "MD": "Este", "DC": "Este", "VA": "Este", "WV": "Este", 
    "NC": "Este", "SC": "Este", "GA": "Este", "FL": "Este",

    # --- CENTRO ---
    "IL": "Centro", "IN": "Centro", "IA": "Centro", "MO": "Centro",
    "KS": "Centro", "NE": "Centro", "KY": "Centro", "TN": "Centro", "AR": "Centro",

    # --- SUR ---
    "AL": "Sur", "MS": "Sur", "LA": "Sur", "TX": "Sur", "OK": "Sur", 
    "NM": "Sur", "AZ": "Sur",

    # --- OESTE ---
    "CO": "Oeste", "UT": "Oeste", "NV": "Oeste", "CA": "Oeste", 
    "OR": "Oeste", "WA": "Oeste", "ID": "Oeste", "MT": "Oeste", "WY": "Oeste", "AK": "Oeste", "HI": "Oeste"

}

In [24]:
base_cox['Abr'] = base_cox['Estado'].map(state_abbr)
base_cox['region'] = base_cox['Abr'].map(state_region_map)
base_cox =  base_cox.groupby(["region", "Trim"])[['TasaDesempleo',"GDP", "Personal_Income"]].mean().reset_index()
base_cox

,region,Trim,TasaDesempleo,GDP,Personal_Income
0,Centro,1990Q1,5.548148,NaN,79360.255556
1,Centro,1990Q2,4.937037,NaN,80676.633333
2,Centro,1990Q3,5.048148,NaN,81787.411111
3,Centro,1990Q4,5.125926,NaN,82319.733333
4,Centro,1991Q1,6.503704,NaN,82301.033333
...,...,...,...,...,...
710,Sur,2024Q3,3.894444,590238.583333,616170.183333
711,Sur,2024Q4,3.627778,593509.066667,622919.400000
712,Sur,2025Q1,3.855556,589457.400000,633244.200000
713,Sur,2025Q2,3.700000,597093.750000,642805.633333


In [25]:
base_cox['U_3m'] = base_cox.groupby('region')['TasaDesempleo'].transform(lambda x: x.rolling(3, min_periods = 3).mean().round(2))
base_cox['U_min12']= base_cox.groupby('region')['U_3m'].transform(lambda x: x.rolling(12, min_periods = 12).min().round(2))
base_cox['SahmRule'] = (base_cox['U_3m']-base_cox['U_min12']).round(2)
base_cox['Dummy'] = (base_cox['SahmRule'] >=0.5).astype(int)
base_cox

,region,Trim,TasaDesempleo,GDP,Personal_Income,U_3m,U_min12,SahmRule,Dummy
0,Centro,1990Q1,5.548148,NaN,79360.255556,NaN,NaN,NaN,0
1,Centro,1990Q2,4.937037,NaN,80676.633333,NaN,NaN,NaN,0
2,Centro,1990Q3,5.048148,NaN,81787.411111,5.18,NaN,NaN,0
3,Centro,1990Q4,5.125926,NaN,82319.733333,5.04,NaN,NaN,0
4,Centro,1991Q1,6.503704,NaN,82301.033333,5.56,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
710,Sur,2024Q3,3.894444,590238.583333,616170.183333,3.59,3.25,0.34,0
711,Sur,2024Q4,3.627778,593509.066667,622919.400000,3.64,3.25,0.39,0
712,Sur,2025Q1,3.855556,589457.400000,633244.200000,3.79,3.25,0.54,1
713,Sur,2025Q2,3.700000,597093.750000,642805.633333,3.73,3.25,0.48,0


In [26]:
base_cox

,region,Trim,TasaDesempleo,GDP,Personal_Income,U_3m,U_min12,SahmRule,Dummy
0,Centro,1990Q1,5.548148,NaN,79360.255556,NaN,NaN,NaN,0
1,Centro,1990Q2,4.937037,NaN,80676.633333,NaN,NaN,NaN,0
2,Centro,1990Q3,5.048148,NaN,81787.411111,5.18,NaN,NaN,0
3,Centro,1990Q4,5.125926,NaN,82319.733333,5.04,NaN,NaN,0
4,Centro,1991Q1,6.503704,NaN,82301.033333,5.56,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
710,Sur,2024Q3,3.894444,590238.583333,616170.183333,3.59,3.25,0.34,0
711,Sur,2024Q4,3.627778,593509.066667,622919.400000,3.64,3.25,0.39,0
712,Sur,2025Q1,3.855556,589457.400000,633244.200000,3.79,3.25,0.54,1
713,Sur,2025Q2,3.700000,597093.750000,642805.633333,3.73,3.25,0.48,0


In [27]:
base_cox = base_cox[base_cox['Trim']>='2005-01-01']
base_cox = base_cox[base_cox['Trim']<'2015-01-01']
base_cox['GDP'] = np.log(base_cox['GDP'])
base_cox['Personal_Income']=np.log(base_cox['Personal_Income'])
base_cox['Fecha'] = pd.PeriodIndex(base_cox['Trim'], freq='Q').to_timestamp()
base_cox

,region,Trim,TasaDesempleo,GDP,Personal_Income,U_3m,U_min12,SahmRule,Dummy,Fecha
60,Centro,2005Q1,5.803704,12.441781,11.998768,5.34,4.94,0.40,0,2005-01-01
61,Centro,2005Q2,5.111111,12.445998,12.012820,5.31,4.94,0.37,0,2005-04-01
62,Centro,2005Q3,4.955556,12.452715,12.024955,5.29,4.94,0.35,0,2005-07-01
63,Centro,2005Q4,4.762963,12.454204,12.038727,4.94,4.94,0.00,0,2005-10-01
64,Centro,2006Q1,5.155556,12.466417,12.063999,4.96,4.94,0.02,0,2006-01-01
...,...,...,...,...,...,...,...,...,...,...
667,Sur,2013Q4,6.355556,12.964845,12.752319,6.82,6.82,0.00,0,2013-10-01
668,Sur,2014Q1,6.461111,12.959173,12.777669,6.64,6.64,0.00,0,2014-01-01
669,Sur,2014Q2,6.083333,12.976422,12.794534,6.30,6.30,0.00,0,2014-04-01
670,Sur,2014Q3,6.444444,12.991227,12.805515,6.33,6.30,0.03,0,2014-07-01


In [37]:
base_cox_d = base_cox[base_cox['Dummy'] == 1]
first_unemp = base_cox_d.groupby('region')['Fecha'].min().reset_index()
first_unemp = first_unemp.rename(columns={'Fecha':'start_date'})
first_unemp

,region,start_date
0,Centro,2008-07-01
1,Este,2008-04-01
2,Norte,2008-04-01
3,Oeste,2008-04-01
4,Sur,2008-07-01


In [ ]:
base_cox_sr = base_cox[base_cox['Dummy'] == 0]
df_merge = pd.merge(base_cox_sr, first_unemp, on='region', how='left')
df_merge = df_merge[df_merge['Fecha'] > df_merge['start_date']]
df_merge

,region,Trim,TasaDesempleo,GDP,Personal_Income,U_3m,U_min12,SahmRule,Dummy,Fecha,start_date
14,Centro,2012Q1,7.529630,12.522553,12.277307,7.50,7.50,0.00,0,2012-01-01,2008-07-01
15,Centro,2012Q2,6.788889,12.524624,12.286464,7.15,7.15,0.00,0,2012-04-01,2008-07-01
16,Centro,2012Q3,6.892593,12.518104,12.280721,7.07,7.07,0.00,0,2012-07-01,2008-07-01
17,Centro,2012Q4,6.474074,12.512089,12.308793,6.72,6.72,0.00,0,2012-10-01,2008-07-01
18,Centro,2013Q1,7.337037,12.531730,12.295012,6.90,6.72,0.18,0,2013-01-01,2008-07-01
19,Centro,2013Q2,6.670370,12.528682,12.300434,6.83,6.72,0.11,0,2013-04-01,2008-07-01
20,Centro,2013Q3,6.581481,12.535011,12.303410,6.86,6.72,0.14,0,2013-07-01,2008-07-01
21,Centro,2013Q4,5.974074,12.540781,12.305159,6.41,6.41,0.00,0,2013-10-01,2008-07-01
22,Centro,2014Q1,6.488889,12.538153,12.319806,6.35,6.35,0.00,0,2014-01-01,2008-07-01
23,Centro,2014Q2,5.503704,12.553782,12.335007,5.99,5.99,0.00,0,2014-04-01,2008-07-01


In [ ]:
#Regreso de Recesion 

first_back = df_merge.groupby('region')['Fecha'].min().reset_index()
first_back = first_back.rename(columns={'Fecha':'end_date'})
first_back  

,region,end_date
0,Centro,2012-01-01
1,Este,2012-04-01
2,Norte,2012-01-01
3,Oeste,2012-04-01
4,Sur,2012-04-01


In [65]:
df_duration = pd.merge(first_unemp, first_back, on='region', how='left')
df_duration['end_date'] = df_duration['end_date'].fillna(base_cox['Fecha'].max())
df_duration['duration_quarters'] = ((df_duration['end_date'] - df_duration['start_date']).dt.days / 30 / 3).round(2)
df_duration['event'] = (df_duration['end_date'] != base_cox['Fecha'].max()).astype(int)
df_duration

,region,start_date,end_date,duration_quarters,event
0,Centro,2008-07-01,2012-01-01,14.21,1
1,Este,2008-04-01,2012-04-01,16.23,1
2,Norte,2008-04-01,2012-01-01,15.22,1
3,Oeste,2008-04-01,2012-04-01,16.23,1
4,Sur,2008-07-01,2012-04-01,15.22,1


In [66]:
df_covars = base_cox.groupby('region')[['GDP', 'Personal_Income']].mean().reset_index()
df_covars
df_final = pd.merge(df_duration, df_covars, on='region', how='left')
df_final

,region,start_date,end_date,duration_quarters,event,GDP,Personal_Income
0,Centro,2008-07-01,2012-01-01,14.21,1,12.493620,12.192283
1,Este,2008-04-01,2012-04-01,16.23,1,13.017099,12.774096
2,Norte,2008-04-01,2012-01-01,15.22,1,12.759840,12.494906
3,Oeste,2008-04-01,2012-04-01,16.23,1,12.705061,12.404716
4,Sur,2008-07-01,2012-04-01,15.22,1,12.876164,12.595933


In [67]:
%store base_cox

Stored 'base_cox' (DataFrame)


In [68]:
%store df_final

Stored 'df_final' (DataFrame)
